# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
from impyute.imputation.cs import fast_knn
import plotly
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Load data

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
df_imm = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
feats_imm = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
df_ld_imm = df_imm['Subject ID'].value_counts().to_frame()
df_imm['Is longitudinal?'] = False
df_imm.loc[df_imm['Subject ID'].isin(df_ld_imm.index[df_ld_imm['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df_imm = df_imm.loc[(df_imm['Status'] == 'Control'), :]
df_imm.rename(columns={'Sample_Chronology': 'Time'}, inplace=True)
df_imm['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)
df_imm_ppr = pd.read_excel(f"{path}/data/immuno/models/SImAge/data.xlsx", index_col="sample_id")
ids_imm_ppr = df_imm_ppr.index[df_imm_ppr['Status'] == 'Control'].values
feats_imm_ppr = pd.read_excel(f"{path}/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values

epi_suffix = "_harm"
df_epi = pd.read_excel(f"{path}/pheno.xlsx", index_col="index")
df_epi.index.name = "index"
df_epi.drop(["I64_old", "I1_duplicate"], inplace=True)
df_epi.rename(columns={'Subject_ID': 'Subject ID'}, inplace=True)
df_ld_epi = df_epi['Subject ID'].value_counts().to_frame()
df_epi['Is longitudinal?'] = False
df_epi.loc[df_epi['Subject ID'].isin(df_ld_epi.index[df_ld_epi['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df_epi = df_epi.loc[(df_epi['Status'] == 'Control'), :]
df_epi.rename(columns={'Sample_Chronology': 'Time'}, inplace=True)
df_epi['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)
ids_epi_ppr = pd.read_excel(f"{path}/data/GSE234461/samples.xlsx", index_col=0).index.values

path_save = f"{path}/special/059_imm_data_selection"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

## Get immunology data with nans 

In [ ]:
files = [
    "Aging L, Q, H, I",
    "Aging-Covid_05.01.2022",
    "Aging-Covid-05.05.22",
    "Covid_results_02_2021",
    "Covid-25.11.20",
    "MULTIPLEX_20_11_2020_ AGING",
    "Yakutiya + TR",
    "Мультиплекс_Agind&Covid",
]
df_imm_genes = pd.read_excel(f"{path}/data/immuno/immuno_markers_genes.xlsx")
dict_imm_genes = dict(zip(df_imm_genes['immuno_marker'], df_imm_genes['gene']))

dfs_files = []
nans_by_features = {}
for file in files:
    df_file = pd.read_excel(f"{path}/data/immuno/files/processed/{file}.xlsx", index_col="Sample")
    df_file.rename(columns=dict_imm_genes, inplace=True)
    df_file = df_file.loc[:, feats_imm]

    # duplicates processing
    if file == "MULTIPLEX_20_11_2020_ AGING":
        df_file_doubled_unique = df_file.loc[~df_file.index.duplicated(keep=False), :]
        df_file_doubled_1 = df_file.loc[df_file.index.duplicated(keep='first'), :]
        df_file_doubled_2 = df_file.loc[df_file.index.duplicated(keep='last'), :]
        df_file_duplicates_final = pd.concat([df_file_doubled_2, df_file_doubled_unique], axis=0)
        df_file = df_file_duplicates_final
    df_file_duplicates = df_file.loc[df_file.index.duplicated(keep=False), :]
    if df_file_duplicates.shape[0] > 0:
        print(df_file_duplicates.index)
    
    for feat in df_file:
        nan_vals = set(df_file.loc[df_file[feat].astype(str).str.contains(r'^([<>].*)$', regex=True), feat].values)
        if len(nan_vals) > 0:
            for nv in nan_vals:
                if feat in nans_by_features:
                    nans_by_features[feat].add(nv)
                else:
                    nans_by_features[feat] = {nv}
    
    dfs_files.append(df_file)

print(nans_by_features)

df_imm_w_nans = pd.concat(dfs_files, verify_integrity=False)
df_imm_w_nans.index = df_imm_w_nans.index.map(str)
df_imm_w_nans = df_imm_w_nans.loc[df_imm.index.values, :]
df_imm_w_nans.replace(r'^([\<].*)$', 'NaN', inplace=True, regex=True)
for feat in feats_imm:
    ids_imputed_above = df_imm_w_nans.index[df_imm_w_nans[feat].astype(str).str.contains('>')]
    df_imm_w_nans.loc[ids_imputed_above, feat] = df_imm.loc[ids_imputed_above, feat]
df_imm_w_nans = df_imm_w_nans.apply(pd.to_numeric, errors='coerce')
df_imm_w_nans.to_excel(f"{path_save}/df_imm_w_nans.xlsx", index_label="Index")

# Checking values
df_diff = df_imm.loc[df_imm.index.values, feats_imm] - df_imm_w_nans.loc[df_imm.index.values, feats_imm]
df_diff = df_diff.fillna(0.0).values
max_diff = df_diff.flat[np.abs(df_diff).argmax()]  
print(f"max_diff: {max_diff}")

# Yakutia data threshold processing

## Replace NaN (threshold) values in Yakutia close to Central region samples

In [ ]:
pathlib.Path(f"{path_save}/yakutia_imputation/").mkdir(parents=True, exist_ok=True)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

df_imm_imptd = df_imm_w_nans.loc[:, feats_imm].copy()
ids_imm_non_yakutia = df_imm.index[df_imm['Region'] != 'Yakutia'].values
ids_imm_yakutia = df_imm.index[df_imm['Region'] == 'Yakutia'].values
df_imm_imptd.loc[ids_imm_non_yakutia, feats_imm] = df_imm.loc[ids_imm_non_yakutia, feats_imm]
df_imm_imptd.loc[:, feats_imm] = df_imm_imptd.loc[:, feats_imm].astype('float')
imptd_values = fast_knn(df_imm_imptd.loc[:, feats_imm].values)
df_imm_imptd.loc[:, feats_imm] = imptd_values

# Checking values
df_diff = df_imm.loc[ids_imm_non_yakutia, feats_imm] - df_imm_imptd.loc[ids_imm_non_yakutia, feats_imm]
max_diff = df_diff.values.flat[np.abs(df_diff.values).argmax()]  
print(f"max_diff: {max_diff}")

# Replace imputed values in Yakutia to the closest threshold values in central region
for feat in feats_imm:
    srs_feat_base = df_imm_w_nans.loc[ids_imm_non_yakutia, feat].isna()
    ids_feat_base = srs_feat_base.index[srs_feat_base == True].values
    if len(ids_feat_base) > 0:
        feat_base_vals = df_imm.loc[ids_feat_base, feat].unique()
        srs_feat_trgt = df_imm_w_nans.loc[ids_imm_yakutia, feat].isna()
        ids_feat_trgt = srs_feat_trgt.index[srs_feat_trgt == True].values
        for id_trgt in ids_feat_trgt:
            df_imm_imptd.at[id_trgt, feat] = find_nearest(feat_base_vals, df_imm_imptd.at[id_trgt, feat])

df_diff = df_imm.loc[df_imm.index.values, feats_imm] - df_imm_imptd.loc[df_imm.index.values, feats_imm]
max_diff = df_diff.values.flat[np.abs(df_diff.values).argmax()]  
print(f"max_diff: {max_diff}")

df_imm.loc[ids_imm_yakutia, feats_imm] = df_imm_imptd.loc[ids_imm_yakutia, feats_imm]

# Replace original immunology data with imputed data for Yakuts thresholds

In [ ]:
df_imm_ori = pd.read_excel(f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
df_imm_ori.loc[df_imm.index.values, feats_imm] = df_imm.loc[df_imm.index.values, feats_imm]
df_diff = df_imm_ori.loc[df_imm.index.values, feats_imm] - df_imm.loc[df_imm.index.values, feats_imm]
max_diff = df_diff.values.flat[np.abs(df_diff.values).argmax()]
print(f"Compare imputed data with original data: {max_diff}")
for f in feats_imm:
    df_imm_ori[f"{f}_log"] = np.log(df_imm_ori[f"{f}"])

## Collect new SImAge results

In [ ]:
df_simage = pd.read_excel(f"{path_save}/yakutia_imputation/SImAge_result.xlsx", index_col=0)
df_imm_ori['SImAge old'] = df_imm_ori['SImAge']
df_imm_ori['SImAge acceleration old'] = df_imm_ori['SImAge acceleration']
df_imm_ori['|SImAge acceleration| old'] = df_imm_ori['|SImAge acceleration|']
df_imm_ori.loc[df_imm_ori.index.values, 'SImAge'] = df_simage.loc[df_imm_ori.index.values, 'Prediction']
df_imm_ori.loc[df_imm_ori.index.values, 'SImAge acceleration'] = df_simage.loc[df_imm_ori.index.values, 'Prediction error']
df_imm_ori.loc[df_imm_ori.index.values, '|SImAge acceleration|'] = df_simage.loc[df_imm_ori.index.values, 'Prediction error abs']
df_imm_ori['|SImAge acceleration| diff'] = df_imm_ori['|SImAge acceleration| old'] - df_imm_ori['|SImAge acceleration|']
df_imm_ori['SImAge paper'] = False
df_imm_ori.loc[ids_imm_ppr, 'SImAge paper'] = True
df_imm_ori.to_excel(f"{path_save}/yakutia_imputation/df_imm_ori.xlsx", index_label="index")